## Setup Environment

In [ ]:
!pip install autoadsorbate
!pip install mace-torch

## Init

In [ ]:
import pandas as pd
import seaborn as sns
from google.colab import files
from ase.io import read, write
from ase.build import fcc211

from mace.calculators import mace_mp
from autoadsorbate import Surface, Fragment


# Optional task

Compare the (211) surfaces of Pd and Cu.

1.    How do these surfaces differ in terms of hydrogen (H) affinity?

2.    How do these surfaces differ in terms of oxygen (O) affinity?

3.    How do these surfaces differ in terms of carbon (C) affinity?

4.    How does proximity to a step edge influence substrate affinity?

All creative approaches to answering these questions are welcome. However, here are a few suggestions to help you get started:

# Example of building structures and running MLIP relaxations

Here is a convenient way to generate periodic slab structures as `ase.Atoms` objects using [ASE](https://wiki.fysik.dtu.dk/ase/). Once the slab is created, the next step is to identify which atoms belong to the surface and how to define individual active sites. For this, we can use AutoAdsorbate—you can find more information on how this is done in the [AutoAdsorbate GitHub repository](https://github.com/basf/autoadsorbate).

Next, we can select surrogate molecules in `SMILES` (`*SMILES`) format that bind via a carbon (C) or oxygen (O) atom—or even just hydrogen (H) itself. For guidance on this process, refer again to the [AutoAdsorbate](https://github.com/basf/autoadsorbate) documentation.

Combining this information allows us to create Slab+Fragment configurations. These configurations can then be relaxed using a universal machine-learned interatomic potential (MLIP), such as the [MACE](https://github.com/ACEsuit/mace) and the universal [foundational model](https://github.com/ACEsuit/mace-foundations). Be sure to record the energies obtained from these relaxed structures.



## Make Slab+Fragment configurations

In [ ]:
slab = fcc211(symbol = 'Cu', size=(6,3,3), vacuum=10)  # any ase.Atoms object
s=Surface(slab, touch_sphere_size=2.7)                 # finding all surface atoms
s.sym_reduce()                                         # keeping only non-identical sites

fragments = [
    Fragment('S1SOC=[O+]1', to_initialize=1), # For each *SMILES we can request a differnet number of conformers
    Fragment('ClC(O)=O', to_initialize=1),    # based on how much conformational complexity we expect.
    Fragment('Cl', to_initialize=1),          # this is the surrogate smiles of an adsorbed H atom.
    ]

out_trj = []
for fragment in fragments:

    overlap_thr=1.6
    if fragment.smile == 'Cl':     # the bond between H and the surface is short, so we can relax the overlap criterion
      overlap_thr = 1.

    out_trj += s.get_populated_sites(
      fragment,                    # Fragment object
      site_index='all',            # a single site can be provided here
      sample_rotation=True,        # rotate the Fragment around the surface-fragment bond?
      mode='heuristic',            # 'all' or 'heuristic', if heuristic surrogate smiles with 'Cl...' will be matched with top sites, etc.
      conformers_per_site_cap=5,   # max number of conformers to sample
      overlap_thr=overlap_thr,     # tolerated bond overlap betwen the surface and fragment
      verbose=True
      )
    print('out_trj has this many structures: ', len(out_trj))

write('./out_trj.xyz', out_trj)
files.download('./out_trj.xyz')

We have the trajectory containing constructed Slab+Fragment configurations ready:

In [ ]:
out_trj = read('out_trj.xyz', index=':')

Now that we have constructed our Slab+Fragment configurations, we can proceed with relaxing these structures to obtain more realistic geometries and evaluate their corresponding energies. Structural relaxation is a crucial step, as it allows the system to reach a local energy minimum, providing more accurate insights into surface interactions and binding characteristics.

To carry out the relaxation using the Atomic Simulation Environment (ASE), we need two main components:

An ASE Calculator:
This defines the method by which atomic energies and forces are computed. In our case, we will use a machine-learned interatomic potential (MLIP) such as the MACE foundational model. The calculator serves as the interface between ASE and the underlying energy model, allowing ASE to query energies and forces for a given atomic configuration.

An ASE Dynamics Driver:
This controls how the atomic positions are updated during the relaxation process. In this workflow, we use the FIRE algorithm. FIRE combines aspects of molecular dynamics and gradient-based minimization, making it well-suited for challenging geometries such as surfaces and adsorbates.

Both of these components need to be properly defined before launching the relaxation job. Below, we will provide examples of how to set up the calculator and the FIRE optimizer, and how to initiate the relaxation loop:

In [ ]:
from ase.optimize import FIRE
from ase.constraints import FixAtoms

calc = mace_mp(device='cpu') # Replace with the desired method

data_traj = []

for i, atoms in enumerate(out_trj[:5]):

  relax_id = f'relaxation_{i}'   # keep track of relaxtion index

  relax_atoms = atoms.copy() # Create a copy of the atoms object to avoid modifying the original
  relax_atoms.calc = calc    # Set up the MACE calculator
  relax_atoms.constraints = FixAtoms(indices=[atom.index for atom in relax_atoms if atom.symbol == 'Cu']) # Fix Cu atoms

  relax_atoms.info['mlip_energy_step0'] = relax_atoms.get_potential_energy()

  dyn = FIRE(relax_atoms)    # Optimize the structure using FIRE
  dyn.run(fmax=3.)           # We would consider a structure "good enough" fmax=0.05, and properly relaxed fmax=0.01

  relax_atoms.info.update(
      **{'mlip_energy': relax_atoms.get_potential_energy(), 'relax_id': relax_id} # log info of relaxed structure
  )

  data_traj.append(relax_atoms) # relaxed atoms


# Now 'data_traj' contains the optimized structures.
# You can save them or further process them.
write('./data_traj.xyz', data_traj)
files.download('./data_traj.xyz')


Now we're ready to dive into the data and see what insights it holds!

One particularly convenient feature is how easily we can extract the `atoms.info` dictionary from each structure, collect them into a list, and convert that list into a Pandas DataFrame. This structure makes it straightforward to analyze trends, filter results, and generate plots for visualization.

In [ ]:
data_traj = read('./data_traj.xyz', index=':')
data = []
for a in data_traj:
  atoms = a.copy()
  _info = atoms.info.pop('adsorbate_info')
  __info = _info.pop('site')
  # __info = atoms.info.pop('site')
  data.append({**_info, **__info, **atoms.info})

df = pd.DataFrame(data)

In [ ]:
df

In [ ]:
sns.scatterplot(df, y='mlip_energy', x='relax_id', hue='smiles')